In [1]:
import os, time

In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
embedding_endpoint = 'https://grit-llm-samouelian-edb-ai.apps.ai-dev01.kni.syseng.devcluster.openshift.com/v1/embeddings'
model_name = 'grit-llm'
number_data_rows = 1000

In [4]:
#Evaluate baseline (direct REST calls to vllm) for non-batch use case

In [5]:
curl_setup(embedding_endpoint)
connect_db()

In [6]:
#Compute time for 1k embeddings

In [7]:
sql = f"select description from film limit {number_data_rows}"
results = execute_sql_results(sql)

In [8]:
test_description = "Starting batch embedding test using remote vllm model with curl"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
descriptions = list()
for index, film in enumerate(results):
    descriptions.append(film[0])

batch_size = 1000
start_offset = 0
end_offset = batch_size
embeddings = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    embeddings = embeddings + curl_get_embedding(model_name, descriptions[start_offset:end_offset])
    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(embeddings) == index+1
for e in embeddings:
    assert(len(e) == 4096)

toc = time.perf_counter()
print("")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch embedding test using remote vllm model with curl
---------------------------------------------------------------
Embeddings computed ... 0 1000 
Time to compute 1000 embeddings: 9.8557 seconds
---------------------------------------------------------------


In [ ]:
close_db()